In [1]:
# -*- coding: utf-8 -*
import twitter
import csv
import requests
from urllib2 import urlopen
import pandas as pd
from datetime import datetime, timedelta
from dateutil import tz
import numpy as np
import xml.etree.ElementTree as ET

In [27]:
tree = ET.ElementTree(file=urlopen('https://roem.ru/rss/roem-all-news.xml'))
root = tree.getroot()
root.tag, root.attrib

('rss', {'version': '2.0'})

In [77]:
class RSSLoader:
    
    def __init__(self):
        #self._pages = ["https://roem.ru/rss/roem-all-news.xml", "http://lifenews.ru/xml/feed.xml", "http://www.forbes.ru/newrss.xml", "http://rg.ru/xml/index.xml", "http://www.vesti.ru/vesti.rss", "http://lenta.ru/rss", "http://ria.ru/export/rss2/index.xml"]
        self._pages = ["https://roem.ru/rss/roem-all-news.xml", "http://lifenews.ru/xml/feed.xml", "http://www.forbes.ru/newrss.xml", "http://www.vesti.ru/vesti.rss", "http://lenta.ru/rss", "http://ria.ru/export/rss2/index.xml"]
        self._month_dict = {"Jan":"1", "Feb":"2", "Mar":"3", "Apr":"4", "May":"5", "Jun":"6", "Jul":"7", "Aug":"8", "Sep":"9", "Oct":"10", "Nov":"11", "Dec":"12"}
        
        
        self._UTC_TIME_ZONE = tz.gettz('Europe/London')
        self._MOSCOW_TIME_ZONE = tz.gettz('Europe/Moscow')
        self._RATE_LIMIT = "[{u'message': u'Rate limit exceeded', u'code': 88}]"
        
        
    def _parse_date(self, date):

        """

        :param date: str, дата в формате - "Fri, 04 Dec 2015 14:45:39 +0000"
        :return: str, дата в человеческом, но буржуйском формате, да еще и в Московском часовом поясе
        """
        date = date.replace(",", "")
        date_array = date.split(' ')
        
        time = date_array[4]
        day = int(date_array[1])
        month = self._month_dict[date_array[2]]
        year = date_array[3]

        date = str(year)+"-"+str(month)+"-"+str(day)+" "+time
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        utc_date = date.replace(tzinfo=self._UTC_TIME_ZONE)
        moscow_date = utc_date.astimezone(self._MOSCOW_TIME_ZONE)

        # Обрезаем зону
        date = str(moscow_date).split("+")[0]
        
        # Обрезаем секунды
        splited = date.split(":")
        date = splited[0]+":"+splited[1]
        return date
    
    
    def _get_type(self, url):
        url = url.replace("www", "")
        url = url.split("://")[1]
        url = url.split("/")[0]
        return url
        
        
    def get_news_array(self):
        news_array = []
        for url in self._pages:
            news_type = self._get_type(url)
            print url
            
            tree = ET.ElementTree(file=urlopen(url))
            root = tree.getroot()
            for i in root.iter('item'):
                link = i.find('link').text
                title = i.find('title').text
                date = i.find('pubDate').text
                date = self._parse_date(date)
                
                news_info =  {
                    "title": title,
                    "date": date, 
                    "url": link, 
                    "type": news_type
                }
                
                news_array.append(news_info)
            print date
        
        return news_array

In [80]:
s = RSSLoader()
df = pd.DataFrame(s.get_news_array())

https://roem.ru/rss/roem-all-news.xml
2015-11-30 19:22
http://lifenews.ru/xml/feed.xml
2015-12-04 15:02
http://www.forbes.ru/newrss.xml
2015-12-04 20:46
http://www.vesti.ru/vesti.rss
2015-12-05 17:46
http://lenta.ru/rss
2015-12-04 12:54
http://ria.ru/export/rss2/index.xml
2015-12-05 16:58


In [83]:
s = df[:200]

In [84]:
df = df.append(s)

In [92]:
g=pd.DataFrame(df.groupby("url")["url"].count())
g.columns = ["url_count"]
g.reset_index(inplace=True)
df = df.merge(g, on="url", left_index=True, right_index=False)

In [100]:
len(df[df["url"]=="http://lenta.ru/news/2015/12/04/pantone/"])

1

In [30]:
for i in root.iter('item'):
    print i.find('link').text
    print i.find('title').text
    print i.find('pubDate').text

https://roem.ru/04-12-2015/214353/cubeline-streaming/
Как стриминг приложений в Google изменит рекламный рынок?
Fri, 04 Dec 2015 14:45:39 +0000
https://roem.ru/04-12-2015/214354/and-that-s-where-the-beginning-of-the-end-begun/
Говорят, на выходе из Яндекс.Новостей — все плохо, а «Ведомости» — не написавшее про Чайку гос-СМИ
Fri, 04 Dec 2015 14:29:33 +0000
https://roem.ru/04-12-2015/214351/ganin-yamail/
Кадры: Отвечавший за развитие Яндекс.Почты Егор Ганин пришел в Mail.ru Group
Fri, 04 Dec 2015 10:38:34 +0000
https://roem.ru/04-12-2015/214298/rbk-ny/
“Большой передел медиарынка, и меньший — телекома” — год глазами отраслевых журналистов РБК
Fri, 04 Dec 2015 09:52:01 +0000
https://roem.ru/04-12-2015/214334/app-play-nds/
Депутаты предложили обложить НДС покупки в App Store и Google Play
Fri, 04 Dec 2015 09:34:42 +0000
https://roem.ru/04-12-2015/214330/uber-dont-like-kids/
Открытка компании: Почему Uber не хочет возить детей?
Fri, 04 Dec 2015 08:45:50 +0000
https://roem.ru/04-12-2015/2143

In [ ]:
root = ET.fromstring(source.encode('utf-8'))#.getroot()

In [ ]:
return {
    "title": title, 
    "views": view, 
    "comments": comment, 
    "tags": tag_list, 
    "date": date, 
    "url": link, 
    "type": news_type, 
    "source": source
}